<a href="https://colab.research.google.com/github/Sontakkepratham/tourism-ai-app/blob/main/tourist_app_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os

DATA_PATH = "/content/sample_data"

files = {
    "transaction": "/content/Transaction.xlsx",
    "user" : "/content/User.xlsx",
    "city": "/content/City.xlsx",
    "country": "/content/Country.xlsx",
    "region": "/content/Region.xlsx",
    "continent": "/content/Continent.xlsx",
    "item": "/content/Item.xlsx",
    "updated_item": "/content/Updated_Item.xlsx",
    "type": "/content/Type.xlsx",
    "mode": "/content/Mode.xlsx"
}

dfs = {}
for name, file in files.items():
  path = os.path.join(DATA_PATH, file)
  dfs[name] = pd.read_excel(path)
  print(f"{name.upper()} SHAPE:", dfs[name].shape)

TRANSACTION SHAPE: (52930, 7)
USER SHAPE: (33530, 5)
CITY SHAPE: (9143, 3)
COUNTRY SHAPE: (165, 3)
REGION SHAPE: (22, 3)
CONTINENT SHAPE: (6, 2)
ITEM SHAPE: (30, 5)
UPDATED_ITEM SHAPE: (1698, 5)
TYPE SHAPE: (17, 2)
MODE SHAPE: (6, 2)


In [2]:
for name, df in dfs.items():
  print("\n", "="*60)
  print(f"{name.upper()}")
  print(df.info())
  print(df.head(3))


TRANSACTION
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52930 entries, 0 to 52929
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   TransactionId  52930 non-null  int64
 1   UserId         52930 non-null  int64
 2   VisitYear      52930 non-null  int64
 3   VisitMonth     52930 non-null  int64
 4   VisitMode      52930 non-null  int64
 5   AttractionId   52930 non-null  int64
 6   Rating         52930 non-null  int64
dtypes: int64(7)
memory usage: 2.8 MB
None
   TransactionId  UserId  VisitYear  VisitMonth  VisitMode  AttractionId  \
0              3   70456       2022          10          2           640   
1              8    7567       2022          10          4           640   
2              9   79069       2022          10          3           640   

   Rating  
0       5  
1       5  
2       5  

USER
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33530 entries, 0 to 33529
Data columns (total 5 c

In [3]:
dfs["updated_item"]["AttractionTypeId"].unique()[:20]

array([13, 63, 76, 72, 93, 92, 61, 64, 82, 91, 84, 19, 34, 10, 2, 45, 44,
       'Temple', 'Beach', 'Park'], dtype=object)

In [4]:
dfs["updated_item"]["AttractionTypeId_clean"] = (pd.to_numeric(dfs["updated_item"]["AttractionTypeId"], errors="coerce"))

In [5]:
updated_item = dfs["updated_item"].merge(
    dfs["type"],
    left_on="AttractionTypeId_clean",
    right_on="AttractionTypeId",
    how="left"
)

In [6]:
updated_item["AttractionType"] = updated_item["AttractionType"].fillna("Unknown")

In [7]:
dfs["updated_item"] = updated_item

In [8]:
dfs["updated_item"]["AttractionType"].value_counts().head(10)

,count
AttractionType,
Unknown,1668
Beaches,6
Points of Interest & Landmarks,3
Nature & Wildlife Areas,2
Waterfalls,2
Religious Sites,2
Volcanos,2
National Parks,2
Ancient Ruins,2


In [9]:
(dfs["updated_item"]["AttractionType"]=="Unknown").sum()

np.int64(1668)

In [10]:
df.columns

Index(['VisitModeId', 'VisitMode'], dtype='object')

In [11]:
print(df.columns)

Index(['VisitModeId', 'VisitMode'], dtype='object')


In [12]:
geo = dfs["city"]
geo = geo.merge(dfs["country"], on="CountryId", how="left")
geo = geo.merge(dfs["region"], on="RegionId", how="left")
geo = geo.merge(dfs["continent"], on="ContinentId", how="left")

geo.columns

Index(['CityId', 'CityName', 'CountryId', 'Country', 'RegionId', 'Region',
       'ContinentId', 'Continent'],
      dtype='object')

In [13]:
df = dfs["transaction"].merge(dfs["user"], on="UserId", how="left")
df = df.merge(geo, on="CityId", how="left")

In [14]:
dfs["updated_item"] = dfs["updated_item"][[
    "AttractionId",
    "AttractionCityId",
    "Attraction",
    "AttractionType"
]]

In [15]:
dfs["updated_item"].columns.tolist()

['AttractionId', 'AttractionCityId', 'Attraction', 'AttractionType']

In [16]:
dfs["updated_item"].columns

Index(['AttractionId', 'AttractionCityId', 'Attraction', 'AttractionType'], dtype='object')

In [17]:
df = df.merge(
    dfs["updated_item"],
    on="AttractionId",
    how="left"
)

In [18]:
df[["Attraction", "AttractionType"]].head()

,Attraction,AttractionType
0,Sacred Monkey Forest Sanctuary,Nature & Wildlife Areas
1,Sacred Monkey Forest Sanctuary,Nature & Wildlife Areas
2,Sacred Monkey Forest Sanctuary,Nature & Wildlife Areas
3,Sacred Monkey Forest Sanctuary,Nature & Wildlife Areas
4,Sacred Monkey Forest Sanctuary,Nature & Wildlife Areas


In [19]:
df.columns

Index(['TransactionId', 'UserId', 'VisitYear', 'VisitMonth', 'VisitMode',
       'AttractionId', 'Rating', 'ContinentId_x', 'RegionId_x', 'CountryId_x',
       'CityId', 'CityName', 'CountryId_y', 'Country', 'RegionId_y', 'Region',
       'ContinentId_y', 'Continent', 'AttractionCityId', 'Attraction',
       'AttractionType'],
      dtype='object')

In [20]:
df = df.rename(columns={"VisitMode_y":"VisitModeLabel"})

In [21]:
df.columns.tolist()

['TransactionId',
 'UserId',
 'VisitYear',
 'VisitMonth',
 'VisitMode',
 'AttractionId',
 'Rating',
 'ContinentId_x',
 'RegionId_x',
 'CountryId_x',
 'CityId',
 'CityName',
 'CountryId_y',
 'Country',
 'RegionId_y',
 'Region',
 'ContinentId_y',
 'Continent',
 'AttractionCityId',
 'Attraction',
 'AttractionType']

In [22]:
[col for col in df.columns if "VisitMode" in col]

['VisitMode']

In [23]:
df["AttractionType"].value_counts().head(10)

,count
AttractionType,
Nature & Wildlife Areas,13251
Beaches,10917
Religious Sites,6711
Water Parks,6429
Points of Interest & Landmarks,6252
Volcanos,2318
Waterfalls,2245
Flea & Street Markets,1324
History Museums,978


In [24]:
df.groupby("AttractionType")["Rating"].mean().sort_values(ascending=False).head(10)

,Rating
AttractionType,
Water Parks,4.646601
Spas,4.571429
Caverns & Caves,4.503704
National Parks,4.416830
Neighborhoods,4.318182
Nature & Wildlife Areas,4.266848
History Museums,4.256646
Religious Sites,4.207123
Ballets,4.180371


In [25]:
df["VisitMode"].value_counts()

,count
VisitMode,
2,21620
3,15217
4,10945
5,4525
1,623


In [26]:
[col for col in df.columns if "VisitMode" in col]

['VisitMode']

In [27]:
df["Attraction"].value_counts().head(10)

,count
Attraction,
Sacred Monkey Forest Sanctuary,13198
Waterbom Bali,6429
Tegalalang Rice Terrace,5815
Uluwatu Temple,3359
Tanah Lot Temple,3352
Sanur Beach,3044
Seminyak Beach,2914
Kuta Beach - Bali,2765
Merapi Volcano,2235


In [28]:
df["AttractionType"].value_counts().head(10)

,count
AttractionType,
Nature & Wildlife Areas,13251
Beaches,10917
Religious Sites,6711
Water Parks,6429
Points of Interest & Landmarks,6252
Volcanos,2318
Waterfalls,2245
Flea & Street Markets,1324
History Museums,978


In [29]:
df.groupby("AttractionType")["Rating"].mean().sort_values(ascending=False)

,Rating
AttractionType,
Water Parks,4.646601
Spas,4.571429
Caverns & Caves,4.503704
National Parks,4.416830
Neighborhoods,4.318182
Nature & Wildlife Areas,4.266848
History Museums,4.256646
Religious Sites,4.207123
Ballets,4.180371


In [30]:
[col for col in df.columns if "Visit" in col]

['VisitYear', 'VisitMonth', 'VisitMode']

In [31]:
df["VisitMode"].unique()

array([2, 4, 3, 5, 1])

In [32]:
dfs["mode"].head()

,VisitModeId,VisitMode
0,0,-
1,1,Business
2,2,Couples
3,3,Family
4,4,Friends


In [33]:
df = df.merge(
    dfs["mode"],
    left_on="VisitMode",
    right_on="VisitModeId",
    how="left"
)

In [34]:
df.columns

Index(['TransactionId', 'UserId', 'VisitYear', 'VisitMonth', 'VisitMode_x',
       'AttractionId', 'Rating', 'ContinentId_x', 'RegionId_x', 'CountryId_x',
       'CityId', 'CityName', 'CountryId_y', 'Country', 'RegionId_y', 'Region',
       'ContinentId_y', 'Continent', 'AttractionCityId', 'Attraction',
       'AttractionType', 'VisitModeId', 'VisitMode_y'],
      dtype='object')

In [35]:
df[["VisitMode_x","VisitMode_y"]].head()

,VisitMode_x,VisitMode_y
0,2,Couples
1,4,Friends
2,3,Family
3,3,Family
4,2,Couples


In [36]:
df = df.rename(columns={"VisitMode_y":"VisitModeLabel"})
df = df.drop(columns=["VisitMode_x", "VisitModeId"])

In [37]:
df["VisitModeLabel"].unique()

array(['Couples', 'Friends', 'Family', 'Solo', 'Business'], dtype=object)

In [38]:
df["VisitModeLabel"].value_counts()
df["AttractionType"].value_counts().head(10)
df.groupby("AttractionType")["Rating"].mean().sort_values(ascending=False)
df["Rating"].value_counts().sort_index()
df["Attraction"].value_counts().head(10)

,count
Attraction,
Sacred Monkey Forest Sanctuary,13198
Waterbom Bali,6429
Tegalalang Rice Terrace,5815
Uluwatu Temple,3359
Tanah Lot Temple,3352
Sanur Beach,3044
Seminyak Beach,2914
Kuta Beach - Bali,2765
Merapi Volcano,2235


In [39]:
df["VisitModeLabel"].value_counts()

,count
VisitModeLabel,
Couples,21620
Family,15217
Friends,10945
Solo,4525
Business,623


In [40]:
df["AttractionType"].value_counts()

,count
AttractionType,
Nature & Wildlife Areas,13251
Beaches,10917
Religious Sites,6711
Water Parks,6429
Points of Interest & Landmarks,6252
Volcanos,2318
Waterfalls,2245
Flea & Street Markets,1324
History Museums,978


In [41]:
df.groupby("AttractionType")["Rating"].mean().sort_values(ascending=False)

,Rating
AttractionType,
Water Parks,4.646601
Spas,4.571429
Caverns & Caves,4.503704
National Parks,4.416830
Neighborhoods,4.318182
Nature & Wildlife Areas,4.266848
History Museums,4.256646
Religious Sites,4.207123
Ballets,4.180371


In [42]:
df["Rating"].value_counts().sort_index()

,count
Rating,
1,1263
2,2035
3,7730
4,17966
5,23936


In [43]:
attr_pop = df.groupby("AttractionId").size().reset_index(name="attraction_popularity")
df = df.merge(attr_pop, on="AttractionId", how="left")

In [44]:
attr_rating = df.groupby("AttractionId")["Rating"].mean().reset_index(name="attr_avg_rating")
df = df.merge(attr_rating, on="AttractionId", how="left")

In [45]:
user_stats = df.groupby("UserId").agg(
    user_total_visits=("AttractionId", "count"),
    user_avg_rating=("Rating", "mean")
).reset_index()

df = df.merge(user_stats, on="UserId", how="left")

In [46]:
df.head()

,TransactionId,UserId,VisitYear,VisitMonth,AttractionId,Rating,ContinentId_x,RegionId_x,CountryId_x,CityId,...,ContinentId_y,Continent,AttractionCityId,Attraction,AttractionType,VisitModeLabel,attraction_popularity,attr_avg_rating,user_total_visits,user_avg_rating
0,3,70456,2022,10,640,5,5,21,163,4341.0,...,4.0,Australia & Oceania,1,Sacred Monkey Forest Sanctuary,Nature & Wildlife Areas,Couples,13198,4.267086,1,5.0
1,8,7567,2022,10,640,5,2,8,48,464.0,...,2.0,America,1,Sacred Monkey Forest Sanctuary,Nature & Wildlife Areas,Friends,13198,4.267086,1,5.0
2,9,79069,2022,10,640,5,2,9,54,774.0,...,2.0,America,1,Sacred Monkey Forest Sanctuary,Nature & Wildlife Areas,Family,13198,4.267086,1,5.0
3,10,31019,2022,10,640,3,5,17,135,583.0,...,2.0,America,1,Sacred Monkey Forest Sanctuary,Nature & Wildlife Areas,Family,13198,4.267086,2,3.0
4,15,43611,2022,10,640,3,5,21,163,1396.0,...,2.0,America,1,Sacred Monkey Forest Sanctuary,Nature & Wildlife Areas,Couples,13198,4.267086,3,3.0


In [47]:
features = [
    "VisitYear",
    "VisitMonth",
    "attraction_popularity",
    "attr_avg_rating",
    "user_total_visits",
    "user_avg_rating"
]

In [48]:
x = df[features]
y = df["Rating"]

In [49]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

In [50]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(x_train, y_train)

RandomForestRegressor(random_state=42)

In [51]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

features = [
    "VisitYear",
    "VisitMonth",
    "attraction_popularity",
    "attr_avg_rating",
    "user_total_visits",
    "user_avg_rating"
]
x = df[features]
y = df["Rating"]

x_train, x_test, y_train, y_test = train_test_split(
    x, y,
    test_size=0.2,
    random_state=42
)

model = RandomForestRegressor(n_estimators=100, random_state=42)

model.fit(x_train, y_train)

pred = model.predict(x_test)

rmse = np.sqrt(mean_squared_error(y_test, pred))
r2 = r2_score(y_test, pred)

print("RMSE:", rmse)
print("R2 Score:", r2)

RMSE: 0.5430937742157268
R2 Score: 0.6868275893203423


In [52]:
import pandas as pd

feature_importance = pd.DataFrame({
     "feature": features,
     "importance": model.feature_importances_
 })

feature_importance.sort_values(by="importance", ascending=False)

,feature,importance
5,user_avg_rating,0.802207
1,VisitMonth,0.062741
0,VisitYear,0.044856
3,attr_avg_rating,0.035787
4,user_total_visits,0.035403
2,attraction_popularity,0.019006


In [53]:
features_test = [
    "VisitYear",
    "VisitMonth",
    "attraction_popularity",
    "attr_avg_rating",
    "user_total_visits"
]

In [54]:
x  = df[features_test]
y = df["Rating"]

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

model2 = RandomForestRegressor(n_estimators=100, random_state=42)

model2.fit(x_train, y_train)

pred2 = model2.predict(x_test)

rmse2 = np.sqrt(mean_squared_error(y_test, pred2))
r22 = r2_score(y_test, pred2)

print("RMSE without user_avg_rating:", rmse2)
print("R2 without user_avg_rating:", r2)

RMSE without user_avg_rating: 0.9467791960073879
R2 without user_avg_rating: 0.6868275893203423


In [55]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

df["VisitModeEncoded"] = le.fit_transform(df["VisitModeLabel"])

In [56]:
df[["VisitModeLabel", "VisitModeEncoded"]].head()

,VisitModeLabel,VisitModeEncoded
0,Couples,1
1,Friends,3
2,Family,2
3,Family,2
4,Couples,1


In [57]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

le = LabelEncoder()
df["VisitModeEncoder"] = le.fit_transform(df["VisitModeLabel"])

features_clf = [
    "VisitYear",
    "VisitMonth",
    "attraction_popularity",
    "attr_avg_rating",
    "user_total_visits",
    "user_avg_rating"
]

x = df[features_clf]
y = df["VisitModeEncoded"]

from sklearn .model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

clf = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    class_weight="balanced"
)

clf.fit(x_train, y_train)

pred_clf = clf.predict(x_test)

print(classification_report(y_test, pred_clf))


              precision    recall  f1-score   support

           0       0.09      0.31      0.14       130
           1       0.51      0.45      0.47      4290
           2       0.48      0.44      0.46      3079
           3       0.32      0.32      0.32      2171
           4       0.18      0.28      0.22       916

    accuracy                           0.40     10586
   macro avg       0.31      0.36      0.32     10586
weighted avg       0.43      0.40      0.41     10586



In [58]:
user_item_matrix = df.pivot_table(
    index="UserId",
    columns="AttractionId",
    values="Rating"
)
user_item_matrix.head()

AttractionId,369,481,640,650,673,737,748,749,824,841,...,1133,1137,1166,1171,1220,1225,1238,1278,1280,1297
UserId,,,,,,,,,,,,,,,,,,,,,
14,NaN,NaN,4.00,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,NaN,5.0,4.25,NaN,NaN,NaN,5.0,NaN,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
user_item_filled = user_item_matrix.fillna(0)

In [60]:
from sklearn.metrics.pairwise import cosine_similarity

user_similarity = cosine_similarity(user_item_filled)

import pandas as pd

user_similarity_df = pd.DataFrame(
    user_similarity,
    index=user_item_filled.index,
    columns=user_item_filled.index
)
user_similarity_df.head()

UserId,14,16,20,23,25,26,27,28,29,32,...,88168,88170,88175,88177,88178,88179,88185,88187,88189,88190
UserId,,,,,,,,,,,,,,,,,,,,,
14,1.000000,0.603672,0.000000,0.780869,0.0,0.780869,0.624695,0.0,0.624695,0.624695,...,0.624695,0.404029,0.624695,0.624695,0.624695,0.780869,0.327429,0.000000,0.499756,0.780869
16,0.603672,1.000000,0.460165,0.460165,0.0,0.460165,0.391141,0.0,0.391141,0.391141,...,0.391141,0.252975,0.391141,0.391141,0.391141,0.460165,0.578861,0.460165,0.312913,0.460165
20,0.000000,0.460165,1.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.314485,0.000000,0.000000,0.000000
23,0.780869,0.460165,0.000000,1.000000,0.0,1.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.419314,0.000000,0.000000,1.000000
25,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [61]:
def recommend_attractions(user_id, top_n=5):
  similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:11]
  weighted_ratings = user_item_filled.loc[similar_users.index].T.dot(similar_users)
  already_rated = user_item_filled.loc[user_id]
  weighted_ratings = weighted_ratings[already_rated == 0]

  return weighted_ratings.sort_values(ascending=False).head(top_n)

In [62]:
recommend_attractions(14)

,0
AttractionId,
369,0.0
481,0.0
650,0.0
673,0.0
737,0.0


In [63]:
def recommend_attractions(user_id, top_n=5):
  similar_users = user_similarity_df[user_id].sort_values(ascending=False)
  similar_users = similar_users[similar_users.index != user_id]
  similar_users = similar_users.head(20)
  weighted_sum = user_item_filled.loc[similar_users.index].T.dot(similar_users)
  similarity_sum = similar_users.sum()
  if similarity_sum == 0:
    return "No strong similar users found."
  recommendation_scores = weighted_sum / similarity_sum
  already_rated = user_item_filled.loc[user_id]
  recommendation_scores = recommendation_scores[already_rated == 0]

  return recommendation_scores.sort_values(ascending=False).head(top_n)

In [64]:
recommend_attractions(14)

,0
AttractionId,
369,0.0
481,0.0
650,0.0
673,0.0
737,0.0


In [65]:
from sklearn.preprocessing import OneHotEncoder

attraction_features = df[["AttractionId", "AttractionType"]].drop_duplicates()

encoder = OneHotEncoder()

encoded_features = encoder.fit_transform(attraction_features[["AttractionType"]])

import pandas as pd

attraction_feature_matrix = pd.DataFrame(
    encoded_features.toarray(),
    index=attraction_features["AttractionId"]
)

attraction_feature_matrix.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
AttractionId,,,,,,,,,,,,,,,,,
640,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
841,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
673,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
481,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
650,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
from sklearn.metrics.pairwise import cosine_similarity

attraction_similarity = cosine_similarity(attraction_feature_matrix)

attraction_similarity_df = pd.DataFrame(
    attraction_similarity,
    index=attraction_feature_matrix.index,
    columns=attraction_feature_matrix.index
)

In [67]:
def recommend_similar_attractions(attraction_id, top_n=5):
  similar = attraction_similarity_df[attraction_id].sort_values(ascending=False)
  return similar[1:top_n+1]

In [68]:
recommend_similar_attractions(640)

,640
AttractionId,
975,1.0
673,0.0
481,0.0
650,0.0
841,0.0


In [69]:
def hybrid_recommend(user_id, top_n=5):
  collab_scores = recommend_attractions(user_id)
  user_ratings = user_item_filled.loc[user_id]
  if user_ratings.sum() == 0:
    return "User has no ratings."
  favorite_item = user_ratings.idxmax()
  content_scores = attraction_similarity_df[favorite_item]
  hybrid_scores = content_scores.copy()
  if isinstance(collab_scores, pd.Series):
    hybrid_scores = hybrid_scores.add(collab_scores, fill_value=0)
  hybrid_scores = hybrid_scores[user_ratings == 0]

  return hybrid_scores.sort_values(ascending=False).head(top_n)

In [70]:
hybrid_recommend(14)

,0
AttractionId,
1280,1.0
937,1.0
369,0.0
481,0.0
737,0.0


In [71]:
!pip install pyngrok

In [72]:
from pyngrok import ngrok

In [73]:
!streamlit run app.py &

/bin/bash: line 1: streamlit: command not found


In [1]:
import pandas as pd

transaction = pd.read_excel("Transaction.xlsx")
user = pd.read_excel("User.xlsx")
city = pd.read_excel("City.xlsx")
country = pd.read_excel("Country.xlsx")
region = pd.read_excel("Region.xlsx")
continent = pd.read_excel("Continent.xlsx")
updated_item = pd.read_excel("Updated_Item.xlsx")
type_df = pd.read_excel("Type.xlsx")
mode = pd.read_excel("Mode.xlsx")

In [2]:
updated_item.columns

Index(['AttractionId', 'AttractionCityId', 'AttractionTypeId', 'Attraction',
       'AttractionAddress'],
      dtype='object')

In [3]:
geo = city.merge(country, on="CountryId", how="left")
geo = geo.merge(region, on="RegionId", how="left")
geo = geo.merge(continent, on="ContinentId", how="left")

df = transaction.merge(user, on="UserId", how="left")
df = df.merge(geo, on="CityId", how="left")

df = df.merge(updated_item, on="AttractionId", how="left")

df = df.merge(updated_item, on="AttractionId", how="left")

df = df.merge(mode, left_on="VisitMode", right_on="VisitModeId", how="left")

df = df.rename(columns={"VisitMode_y":"VisitModeLabel"})

In [4]:
df.head()

,TransactionId,UserId,VisitYear,VisitMonth,VisitMode_x,AttractionId,Rating,ContinentId_x,RegionId_x,CountryId_x,...,AttractionCityId_x,AttractionTypeId_x,Attraction_x,AttractionAddress_x,AttractionCityId_y,AttractionTypeId_y,Attraction_y,AttractionAddress_y,VisitModeId,VisitModeLabel
0,3,70456,2022,10,2,640,5,5,21,163,...,1,63,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",1,63,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",2,Couples
1,8,7567,2022,10,4,640,5,2,8,48,...,1,63,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",1,63,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",4,Friends
2,9,79069,2022,10,3,640,5,2,9,54,...,1,63,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",1,63,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",3,Family
3,10,31019,2022,10,3,640,3,5,17,135,...,1,63,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",1,63,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",3,Family
4,15,43611,2022,10,2,640,3,5,21,163,...,1,63,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",1,63,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",2,Couples


In [5]:
attr_pop = df.groupby("AttractionId").size().reset_index(name="attraction_popularity")
df = df.merge(attr_pop, on="AttractionId", how="left")

attr_rating = df.groupby("AttractionId")["Rating"].mean().reset_index(name="attr_avg_rating")
df = df.merge(attr_rating, on="AttractionId", how="left")

user_stats = df.groupby("UserId").agg(
    user_total_visits=("AttractionId","count"),
    user_avg_rating=("Rating","mean")
).reset_index()

df = df.merge(user_stats, on="UserId", how="left")

In [8]:
df.columns

Index(['TransactionId', 'UserId', 'VisitYear', 'VisitMonth', 'VisitMode_x',
       'AttractionId', 'Rating', 'ContinentId_x', 'RegionId_x', 'CountryId_x',
       'CityId', 'CityName', 'CountryId_y', 'Country', 'RegionId_y', 'Region',
       'ContinentId_y', 'Continent', 'AttractionCityId_x',
       'AttractionTypeId_x', 'Attraction_x', 'AttractionAddress_x',
       'AttractionCityId_y', 'AttractionTypeId_y', 'Attraction_y',
       'AttractionAddress_y', 'VisitModeId', 'VisitModeLabel',
       'attraction_popularity', 'attr_avg_rating', 'user_total_visits',
       'user_avg_rating', 'VisitModeEncoder'],
      dtype='object')

In [10]:
updated_item.columns

Index(['AttractionId', 'AttractionCityId', 'AttractionTypeId', 'Attraction',
       'AttractionAddress'],
      dtype='object')

In [13]:
updated_item = updated_item.merge(
    type_df,
    on="AttractionTypeId",
    how="left"
)

In [16]:
df = df.merge(
    updated_item[["AttractionId", "AttractionTypeId"]],
    on="AttractionId",
    how="left"
)

In [18]:
df.columns

Index(['TransactionId', 'UserId', 'VisitYear', 'VisitMonth', 'VisitMode_x',
       'AttractionId', 'Rating', 'ContinentId_x', 'RegionId_x', 'CountryId_x',
       'CityId', 'CityName', 'CountryId_y', 'Country', 'RegionId_y', 'Region',
       'ContinentId_y', 'Continent', 'AttractionCityId_x',
       'AttractionTypeId_x', 'Attraction_x', 'AttractionAddress_x',
       'AttractionCityId_y', 'AttractionTypeId_y', 'Attraction_y',
       'AttractionAddress_y', 'VisitModeId', 'VisitModeLabel',
       'attraction_popularity', 'attr_avg_rating', 'user_total_visits',
       'user_avg_rating', 'VisitModeEncoder', 'AttractionTypeId'],
      dtype='object')

In [19]:
updated_item = updated_item.merge(
    type_df,
    on="AttractionTypeId",
    how="left"
)

In [20]:
updated_item.columns

Index(['AttractionId', 'AttractionCityId', 'AttractionTypeId', 'Attraction',
       'AttractionAddress', 'AttractionType_x', 'AttractionType_y',
       'AttractionType'],
      dtype='object')

In [21]:
df = df.merge(
    updated_item[["AttractionId", "AttractionType"]],
    on="AttractionId",
    how="left"
)

In [22]:
df.columns

Index(['TransactionId', 'UserId', 'VisitYear', 'VisitMonth', 'VisitMode_x',
       'AttractionId', 'Rating', 'ContinentId_x', 'RegionId_x', 'CountryId_x',
       'CityId', 'CityName', 'CountryId_y', 'Country', 'RegionId_y', 'Region',
       'ContinentId_y', 'Continent', 'AttractionCityId_x',
       'AttractionTypeId_x', 'Attraction_x', 'AttractionAddress_x',
       'AttractionCityId_y', 'AttractionTypeId_y', 'Attraction_y',
       'AttractionAddress_y', 'VisitModeId', 'VisitModeLabel',
       'attraction_popularity', 'attr_avg_rating', 'user_total_visits',
       'user_avg_rating', 'VisitModeEncoder', 'AttractionTypeId',
       'AttractionType'],
      dtype='object')

In [23]:
df = df.drop(columns=[
    "VisitMode_x",
    "AttractionCityId_x",
    "AttractionCityId_y",
    "AttractionTypeId_x",
    "AttractionTypeId_y",
    "AttractionAddress_x",
    "AttractionAddress_y"
], errors="ignore")

In [24]:
df.columns

Index(['TransactionId', 'UserId', 'VisitYear', 'VisitMonth', 'AttractionId',
       'Rating', 'ContinentId_x', 'RegionId_x', 'CountryId_x', 'CityId',
       'CityName', 'CountryId_y', 'Country', 'RegionId_y', 'Region',
       'ContinentId_y', 'Continent', 'Attraction_x', 'Attraction_y',
       'VisitModeId', 'VisitModeLabel', 'attraction_popularity',
       'attr_avg_rating', 'user_total_visits', 'user_avg_rating',
       'VisitModeEncoder', 'AttractionTypeId', 'AttractionType'],
      dtype='object')

In [29]:
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

le = LabelEncoder()
df["VisitModeEncoded"] = le.fit_transform(df["VisitModeLabel"])

features_clf = [
    "VisitYear",
    "VisitMonth",
    "attraction_popularity",
    "attr_avg_rating",
    "user_total_visits",
    "user_avg_rating"
]

x = df[features_clf]
y = df["VisitModeEncoded"]

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

clf = RandomForestClassifier(
    n_estimators=50,
    random_state=42,
    class_weight="balanced"
)

clf.fit(x_train, y_train)

RandomForestClassifier(class_weight='balanced', n_estimators=50,
                       random_state=42)

In [30]:
import pickle

pickle.dump(df, open("data.pkl", "wb"))
pickle.dump(clf, open("clf_model.pkl", "wb"))